In [4]:
import os
import glob
import pandas as pd
import time
from dotenv import load_dotenv
import mimetypes

# Gemini & LangChain & DeepEval
from google import genai
from google.genai import types
from langchain_community.llms import Ollama
from deepeval.metrics import FaithfulnessMetric
from deepeval.test_case import LLMTestCase

/opt/anaconda3/envs/py312_legai/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# [핵심] 우리가 만든 서비스 모듈 (src 폴더가 있어야 함)
# 만약 src 폴더가 없다면, 이 부분은 주석 처리하고 Mock Class를 만들어야 합니다.
try:
    from toxic_detector import ToxicClauseDetector
    from llm_service import LLM_gemini
    from ollama_wrapper import OllamaDeepEvalWrapper
    from law.legal_context import LawContextManager
    from law.precedent_context import PrecedentContextManager
except ImportError:
    print("⚠️ 'src' 모듈을 찾을 수 없습니다. 경로를 확인해주세요.")

In [6]:
# --- 1. 환경 설정 ---
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

IMAGE_DIR = "images/"  # 계약서 이미지가 들어있는 폴더
DATASET_CSV = "contract_dataset.csv"
RESULT_CSV = "model_performance_results.csv"


In [7]:
# --- 2. [수정됨] Gemini Vision을 이용한 이미지 텍스트 추출 ---
def extract_text_with_gemini(image_path, api_key):
    """
    이미지 파일(JPG, PNG 등)을 Gemini에게 보여주고 텍스트만 추출합니다.
    (OCR 대체)
    """
    client = genai.Client(api_key=api_key)
    
    # 파일 MIME 타입 감지
    mime_type, _ = mimetypes.guess_type(image_path)
    if not mime_type:
        mime_type = "image/png" # 기본값

    try:
        with open(image_path, "rb") as f:
            image_bytes = f.read()

        prompt = """
        이 이미지에 있는 근로계약서 조항 텍스트를 있는 그대로 추출해줘.
        조항 사이는 줄바꿈(Enter) 두 번으로 구분해줘.
        설명이나 요약은 하지 말고 텍스트만 출력해.
        """

        response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=[
                types.Content(
                    parts=[
                        types.Part(text=prompt),
                        types.Part(
                            inline_data=types.Blob(
                                mime_type=mime_type,
                                data=image_bytes
                            )
                        )
                    ]
                )
            ]
        )
        return response.text if response.text else ""
        
    except Exception as e:
        print(f"❌ Gemini Vision Error ({os.path.basename(image_path)}): {e}")
        return ""


In [8]:
def build_dataset_from_images(image_dir):
    """이미지 폴더 -> 텍스트 추출 -> DataFrame 생성"""
    print(f"📂 '{image_dir}' 폴더에서 이미지를 스캔합니다...")
    
    # 지원할 이미지 확장자
    exts = ['*.jpg', '*.jpeg', '*.png', '*.webp']
    image_files = []
    for ext in exts:
        image_files.extend(glob.glob(os.path.join(image_dir, ext)))
    
    data = []
    
    if not image_files:
        print("⚠️ 이미지가 없습니다. 테스트를 위해 더미 데이터를 생성합니다.")
        return pd.DataFrame([
            {"file": "dummy.jpg", "clause_id": 1, "text": "제3조 (임금) 월 급여는 200만원으로 하며 포괄임금으로 한다.", "label_toxic": True},
            {"file": "dummy.jpg", "clause_id": 2, "text": "제10조 (근로시간) 09:00~18:00 (휴게 1시간)", "label_toxic": False}
        ])

    print(f"🔍 총 {len(image_files)}개의 이미지를 발견했습니다. Gemini로 텍스트 추출 시작...")
    
    for img_path in image_files:
        # Gemini Vision 호출
        full_text = extract_text_with_gemini(img_path, GEMINI_API_KEY)
        
        # 조항별 분리 (간단히 줄바꿈 2번 기준)
        # 실제로는 '제N조' 정규식으로 자르는 게 더 정확함
        clauses = full_text.split("\n\n")
        
        for i, clause in enumerate(clauses):
            clean_text = clause.strip()
            if len(clean_text) > 10: # 노이즈 제거
                data.append({
                    "file": os.path.basename(img_path),
                    "clause_id": i + 1,
                    "text": clean_text,
                    "label_toxic": None # 정답은 CSV 저장 후 수동 입력 필요
                })
                
    return pd.DataFrame(data)

In [ ]:
# --- 3. 비교 모델 정의 ---

# 1) Base Gemini (No DeepEval)
class BaseGeminiModel:
    def __init__(self):
        self.client = genai.Client(api_key=GEMINI_API_KEY)
    
    def predict(self, text):
        prompt = f"다음 근로계약 조항이 독소조항(근로자에게 불리)인지 판단해. 맞으면 'TOXIC', 아니면 'SAFE'로 시작해.\n\n{text}"
        try:
            res = self.client.models.generate_content(
                model="gemini-2.5-flash", contents=prompt
            )
            return res.text
        except Exception as e:
            return "Error"

# [Case 2] Gemini + RAG (DeepEval Style Prompt)
class GeminiRAGModel:
    def __init__(self):
        self.client = genai.Client(api_key=GEMINI_API_KEY)
        self.law_manager = LawContextManager()
        self.law_manager.initialize_database()
        self.precedent_manager = PrecedentContextManager()
        self.precedent_manager.initialize_database()
    
    def retrieve_context(self, text):
        """RAG: 법령과 판례를 가져옵니다."""
        laws = self.law_manager.search_relevant_laws(text)
        cases = self.precedent_manager.search_relevant_precedents(text, k=1)
        
        law_text = "\n".join(laws) if laws else "관련 법령 없음"
        case_text = cases[0] if cases else "관련 판례 없음"
        
        return f"[관련 법령]\n{law_text}\n\n[관련 판례]\n{case_text}"

    def predict(self, text):
        # RAG Context 검색
        context = self.retrieve_context(text)
        
        prompt = f"""
        [Context]\n{context}\n
        [Input]\n{text}\n
        위 법적 근거를 바탕으로 이 조항이 독소조항인지 판단해. 
        결과는 반드시 'TOXIC' 또는 'SAFE'로 시작해야 해.
        """
        try:
            res = self.client.models.generate_content(
                model="gemini-2.5-flash", contents=prompt
            )
            return res.text
        except Exception as e:
            return "Error"

# 3) Our Framework (SafeSign Logic)
class OurFrameworkModel:
    def __init__(self):
        self.detector = ToxicClauseDetector() # toxic_detector
    
    def predict(self, text):
        # RAG + G-Eval Rubric 적용된 완전체
        res = self.detector.detect(text)
        status = "TOXIC" if res['is_toxic'] else "SAFE"
        return f"[{status}] Score:{res['risk_score']} / Reason:{res['reason']}"

# 4) Ollama Local (On-Device)
class OllamaRAGModel:
    def __init__(self, model_name="llama3"):
        # 1. Ollama 모델 준비
        self.llm = OllamaDeepEvalWrapper(model_name=model_name)
        
        # 2. RAG 도구 준비 (법령, 판례 검색기)
        self.law_manager = LawContextManager()
        self.law_manager.initialize_database()
        
        self.precedent_manager = PrecedentContextManager()
        self.precedent_manager.initialize_database()

        # 3. 평가자(Judge) 준비 - 평가는 똑똑한 Gemini가 담당
        self.evaluator_llm = LLM_gemini(model_name="gemini-2.5-flash")

    def retrieve_context(self, text):
        """RAG: 법령과 판례를 가져옵니다."""
        laws = self.law_manager.search_relevant_laws(text)
        cases = self.precedent_manager.search_relevant_precedents(text, k=1)
        
        law_text = "\n".join(laws) if laws else "관련 법령 없음"
        case_text = cases[0] if cases else "관련 판례 없음"
        
        return f"[관련 법령]\n{law_text}\n\n[관련 판례]\n{case_text}"

    def predict_and_evaluate(self, text):
        # Step 1: Retrieval (RAG)
        context = self.retrieve_context(text)
        
        # Step 2: Generation (Ollama)
        prompt = f"""
        당신은 한국 노동법 전문가입니다. 아래 [법적 근거]를 참고하여 [조항]이 독소조항인지 판단하세요.
        
        [법적 근거]:
        {context}
        
        [조항]:
        {text}
        
        결과를 '판단: [안전/위험]', '이유: ...' 형식으로 한국어로 짧게 답하세요.
        """
        actual_output = self.llm.generate(prompt)
        
        # Step 3: Evaluation (DeepEval Faithfulness)
        # "Ollama가 근거(법령/판례)를 무시하고 헛소리를 했는지" Gemini가 감시합니다.
        try:
            metric = FaithfulnessMetric(
                threshold=0.5, 
                model=self.evaluator_llm, # 심판은 Gemini
                include_reason=True
            )
            test_case = LLMTestCase(
                input=text,
                actual_output=actual_output,
                retrieval_context=[context] # Ollama가 참고했어야 할 정답지
            )
            metric.measure(test_case)
            
            eval_result = f"Faithfulness: {metric.score} ({'Pass' if metric.is_successful() else 'Fail'})"
        except Exception as e:
            eval_result = f"Eval Error: {e}"
            
        return f"{actual_output}\n\n--- [DeepEval 검증] ---\n{eval_result}"

In [ ]:
# --- 4. 실행 루프 ---
def main():
    print("🚀 [Step 1] 데이터 구축 (Gemini Vision OCR)")
    if os.path.exists(DATASET_CSV):
        print(f"   - 기존 데이터 파일({DATASET_CSV})을 로드합니다.")
        df = pd.read_csv(DATASET_CSV)
    else:
        df = build_dataset_from_images(IMAGE_DIR)
        df.to_csv(DATASET_CSV, index=False, encoding="utf-8-sig")
        print(f"   - 데이터 추출 완료 & 저장 ({DATASET_CSV})")

    print(f"\n🚀 [Step 2] 4가지 모델 성능 평가 시작 (총 {len(df)}개 조항)")
    
    # 모델 인스턴스 생성
    models = {
        "Gemini_Base": BaseGeminiModel(),
        # "Gemini_DeepEval": DeepEvalGeminiModel(), # 필요시 주석 해제
        "SafeSign_Framework": OurFrameworkModel(),
        "Ollama_Local": OllamaRAGModel(model_name="LiquidAI/LFM2-8B-A1B-GGUF") 
    }
    
    results = []
    
    for idx, row in df.iterrows():
        text = row['text']
        print(f"\nProcessing ID {idx+1}...")
        
        row_result = row.to_dict()
        
        for name, model in models.items():
            start_time = time.time()
            try:
                output = model.predict(text)
            except Exception as e:
                output = f"Error: {e}"
            end_time = time.time()
            
            row_result[f"{name}_Output"] = output
            row_result[f"{name}_Time"] = round(end_time - start_time, 2)
            
        results.append(row_result)

    # 결과 저장
    res_df = pd.DataFrame(results)
    res_df.to_csv(RESULT_CSV, index=False, encoding="utf-8-sig")
    print(f"\n✅ 평가 완료! 결과 파일: {RESULT_CSV}")
    
    # 간단한 리포트 출력
    print("\n[평균 소요 시간 비교]")
    time_cols = [c for c in res_df.columns if "_Time" in c]
    print(res_df[time_cols].mean())

if __name__ == "__main__":
    main()